Expected columns after merge:  disease_name, symptoms, treatment,drugs, dosage, source (base url only)

In [ ]:
import requests
import time
import random
import csv
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
import os

import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#use google account wmltsang@gmail.com

In [ ]:
os.getcwd()

'/content'

In [ ]:
#load disease_data csv file
# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the CSV file
csv_path1 = os.path.join(current_directory, 'merge_dataset', 'disease_data.csv')

# Read the CSV file into a DataFrame
df1 = pd.read_csv(csv_path1)

# Display the first few rows of the DataFrame
print(df1.columns)

#expected cols: disease_name, symptoms, treatment,drugs, dosage, source
#rename cols
# Rename 'disease' to 'disease_name'
df1.rename(columns={'disease': 'disease_name'}, inplace=True)

# Define the expected columns in df
expected_columns = ['disease_name', 'symptoms', 'treatment', 'drugs', 'dosage', 'source']

# Add missing columns with null values
for col in expected_columns:
    if col not in df1.columns:
        df1[col] = None

# Reorder columns to match df structure
df1 = df1[expected_columns]

Index(['disease', 'description', 'symptoms', 'treatment'], dtype='object')


In [ ]:
df1.head(2)

,disease_name,symptoms,treatment,drugs,dosage,source
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,None,None,None
1,Avian Influenza,Common initial symptoms of the A(H5) and A(H7N...,Treatment of avian influenza is recommended fo...,None,None,None


In [ ]:
#load disease_data_afraicacdc csv file
# Construct the path to the CSV file
csv_path2 = os.path.join(current_directory, 'merge_dataset', 'diseases_data_africacdc.csv')

# Try reading the CSV with a different encoding
try:
    df2 = pd.read_csv(csv_path2, encoding='latin1')  # Try 'latin1' encoding
except UnicodeDecodeError:
    df2 = pd.read_csv(csv_path2, encoding='ISO-8859-1')  # Fallback to 'ISO-8859-1'

df2['Source '] =  df2['Source '].str.replace(r'(https?://[^/]+).*', r'\1', regex=True)
df2.rename(columns={'Disease Name': 'disease_name','Symptoms': 'symptoms','Source ': 'source'}, inplace=True)

# Define the expected columns in df
expected_columns = ['disease_name', 'symptoms', 'treatment', 'drugs', 'dosage', 'source']

# Add missing columns with null values
for col in expected_columns:
    if col not in df2.columns:
        df2[col] = None

# Reorder columns to match df structure
df2 = df2[expected_columns]

# Display the first few rows of the DataFrame
print(df2.info())
df2.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   disease_name  27 non-null     object
 1   symptoms      27 non-null     object
 2   treatment     0 non-null      object
 3   drugs         0 non-null      object
 4   dosage        0 non-null      object
 5   source        27 non-null     object
dtypes: object(6)
memory usage: 1.4+ KB
None


Index(['disease_name', 'symptoms', 'treatment', 'drugs', 'dosage', 'source'], dtype='object')

In [ ]:
df2.head(2)

,disease_name,symptoms,treatment,drugs,dosage,source
0,Cutaneous Anthrax,Small blisters or bumps that may itch. Swellin...,None,None,None,https://africacdc.org
1,Inhalation Anthrax,"Fever and chills. Chest discomfort, shortness ...",None,None,None,https://africacdc.org


In [ ]:
# Concatenate df1 and df2
df = pd.concat([df1, df2], ignore_index=True)

# Display the concatenated DataFrame
df

,disease_name,symptoms,treatment,drugs,dosage,source
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,None,None,None
1,Avian Influenza,Common initial symptoms of the A(H5) and A(H7N...,Treatment of avian influenza is recommended fo...,None,None,None
2,Chikungunya,Prevention and control are usually done by des...,Not Available,None,None,None
3,Cholera,Prevention and control can be achieved by cons...,Not Available,None,None,None
4,Covid 19,Not Available,Not Available,None,None,None
5,Crimean Congo Haemorrhagic Fever,The incubation period depends on the mode of a...,General supportive care with treatment of symp...,None,None,None
6,Dengue Fever,Dengue should be suspected when a high fever (...,There is no specific treatment for dengue feve...,None,None,None
7,Ebola Virus Disease,There is currently no proven cure for Ebola bu...,Not Available,None,None,None
8,Hepatitis B Virus Hbv,"In most cases, Hepatitis B is symptomless. In ...",There is no specific treatment for acute hepat...,None,None,None
9,Hepatitis C Virus,The incubation period for hepatitis C is 2 wee...,There is currently no vaccine for HCV. However...,None,None,None


In [ ]:
#load drugbank_selenium_drugs
# Get the current working directory
current_directory = os.getcwd()

# Construct the path to the CSV file
csv_path3 = os.path.join(current_directory, 'merge_dataset', 'drugbank_drugs_details.csv')

# Read the CSV file into a DataFrame
drug_master = pd.read_csv(csv_path3)

# Display the first few rows of the DataFrame
print(drug_master.columns)
drug_master.info()

#expected cols: disease_name, symptoms, treatment,drugs, dosage, source
#rename cols
# Rename 'disease' to 'disease_name'
drug_master.rename(columns={'Drug Name': 'drugs','Drug URL': 'source','Drug Background':'drug_background','Drug Indication': 'drug_indication'}, inplace=True)

# Drop unwanted col
drug_master = drug_master.drop(columns=['Unnamed: 0'])

#Fix URL
drug_master['source'] =  drug_master['source'].str.replace(r'(https?://[^/]+).*', r'\1', regex=True)

drug_master.head(2)
# # Define the expected columns in df
# expected_columns = ['disease_name', 'symptoms', 'treatment', 'drugs', 'dosage', 'source']

# # Add missing columns with null values
# for col in expected_columns:
#     if col not in df1.columns:
#         df1[col] = None

# # Reorder columns to match df structure
# df1 = df1[expected_columns]

Index(['Unnamed: 0', 'Drug Name', 'Drug URL', 'Drug Background',
       'Drug Indication'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2812 entries, 0 to 2811
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2812 non-null   int64 
 1   Drug Name        2812 non-null   object
 2   Drug URL         2812 non-null   object
 3   Drug Background  2604 non-null   object
 4   Drug Indication  2349 non-null   object
dtypes: int64(1), object(4)
memory usage: 110.0+ KB


,drugs,source,drug_background,drug_indication
0,1-Palmitoyl-2-oleoyl-sn-glycero-3-(phospho-rac...,https://go.drugbank.com,Palmitoyloleoyl-phosphatidylglycerol was a com...,Palmitoyloleoyl-phosphatidylglycerol was a com...
1,"1,2-Benzodiazepine",https://go.drugbank.com,Benzodiazepine is under investigation for the ...,NaN


In [ ]:
###FAILED ATTEMPT , tried to join drug lookup to add drug cols to df but below only get two records back
### Keep the drug col extracted from treatment

#improve ways to get the drug name from treatment, maybe get more than you can and put in lower case can do look up in drug table to
#filter out non known drug name

# Load NLTK's standard English stopwords
nltk_stopwords = set(stopwords.words('english'))

# Function to extract specific drug names from treatment descriptions
def extract_drugs(treatment_text, nltk_stopwords):
    # Regex to capture potential drug names (capitalized words or multi-word names)
    drug_pattern = r'\b([A-Z][a-zA-Z]*(?:\s[A-Z][a-zA-Z]*)*|[a-z]+)\b'

    # Check if the input is None or not a string
    if not isinstance(treatment_text, str):
        return None

#     # Find all matches
    matches = re.findall(drug_pattern, treatment_text)

#     # Filter out NLTK stopwords and medical-specific stopwords
    filtered_words = [word for word in matches if word.lower() not in nltk_stopwords]

#     # Final filter: Only include capitalized words (likely drug names) or known lowercase drug names
    final_drugs = [drug for drug in filtered_words if drug[0].isupper()]

    return ', '.join(final_drugs) if final_drugs else None

# Apply the function to the 'treatment' column
df['drugs'] = df['treatment'].apply(lambda text: extract_drugs(text, nltk_stopwords))

#Notice drugs col values have comma and let split up the value
# Function to split the string and return a list
def split_drugs(drug_string):
    if pd.isna(drug_string):  # Handle NaN values
        return []
    return [drug.strip() for drug in drug_string.split(',')]

# Apply the function to the 'drugs' column
df['drugs'] = df['drugs'].apply(split_drugs)

# Explode the DataFrame to create separate rows for each drug
df = df.explode('drugs')

# # Display the updated DataFrame
df

,disease_name,symptoms,treatment,drugs,dosage,source
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,All Type,None,None
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,Anthrax,None,None
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,Antibiotics,None,None
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,Antitoxin,None,None
0,Anthrax,The symptoms of anthrax depend on the type of ...,All Type of Anthrax can be treated by Antibiot...,Antibiotics,None,None
...,...,...,...,...,...,...
47,Tuberculosis,"cough with sputum and blood, chest pains, weak...",None,NaN,None,https://africacdc.org
48,Yellow Fever\n,"fever, muscle pain, backache, headache, loss o...",None,NaN,None,https://africacdc.org
49,Zika Virus\n,"rash, fever, headache, malaise, muscle and joi...",None,NaN,None,https://africacdc.org
50,Polio,"fever, fatigue, headache, vomiting, stiffness ...",None,NaN,None,https://africacdc.org


In [ ]:
df.to_csv('disease_data.csv', index=False)

In [ ]:
drug_master.to_csv('drug_master.csv', index=False)

In [ ]:
import spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached thinc-8.0.17-cp311-cp311-linux_x86_64.whl
  Using cached wasabi-0.10.1-py3-none-any.whl.metadata (28 kB)
  Using cached pathy-0.11.0-py3-none-any.whl.metadata (16 kB)
  Using cached smart_open-6.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-1.8.2-py3-none-any.whl.metadata (103 kB)
  Using cached pathlib_abc-0.1.1-py

In [ ]:
# Load the biomedical NLP model
nlp = spacy.load('en_core_sci_sm')


NameError: name 'spacy' is not defined

In [ ]:
# Input text
text = “”"Symptoms of anthrax differ, depending on the type of anthrax.
An itchy sore develops that is similar to an insect bite. This sore may blister and form a black ulcer (sore or eschar).
The sore is usually painless, but it is often surrounded by swelling.
A scab often forms, and then dries and falls off within 2 weeks. Complete healing can take longer.“”"

In [ ]:
# Process the text
doc = nlp(text)
# Extract Named Entities (important biomedical terms)
print(“\n**Named Entities:**“)
for ent in doc.ents:
    print(f”{ent.text} ({ent.label_})“)

In [ ]:
# Tokenization
print(“\n**Tokenized Words:**“)
print([token.text for token in doc])
# Part-of-Speech (POS) Tagging
print(“\n**POS Tags:**“)
for token in doc:
    print(f”{token.text} - {token.pos_}“)
# Dependency Parsing
print(“\n**Dependency Parsing:**“)
for token in doc:
    print(f”{token.text} --> {token.dep_} --> {token.head.text}“)